# RAG & Prompt Engineering with gtext

**Learn how to use gtext to create composable, dynamic prompts for AI/LLM applications.**

## What You'll Learn

- Build reusable prompt templates
- Include dynamic context (git diffs, recent commits, code files)
- Create RAG pipelines with versioned prompts
- Integrate with LLM APIs (OpenAI, Anthropic, etc.)

## Why gtext for RAG?

gtext is like a **weaverbird** 🪶 - it weaves together different pieces of content to create a unified whole:

- **Composable**: Break prompts into reusable components
- **Dynamic**: Always include latest context (code, data, etc.)
- **Versionable**: Track prompt changes in git
- **Traceable**: Know exactly what context was used
- **Reproducible**: Same prompt + same context = same result

## Prerequisites

```bash
pip install gtext
```

## Setup

First, let's install gtext and create a sample project structure.

In [ ]:
# Install gtext
!pip install -q gtext

# Create demo project structure
import os
from pathlib import Path

# Create directories
os.makedirs('demo_project/prompts', exist_ok=True)
os.makedirs('demo_project/context', exist_ok=True)
os.makedirs('demo_project/src', exist_ok=True)

print("✅ Demo project structure created!")

## Example 1: Basic Prompt Template

Let's create a simple code review prompt template.

In [ ]:
%%writefile demo_project/prompts/code-review.md.gtext
# Code Review Request

You are an expert code reviewer. Please review the following code changes.

## Changes to Review

```include
cli: git diff --cached
```

## Review Criteria

Please evaluate:
1. **Code Quality**: Is the code clean, readable, and maintainable?
2. **Best Practices**: Does it follow language/framework best practices?
3. **Security**: Are there any security vulnerabilities?
4. **Performance**: Any performance concerns?
5. **Testing**: Is the code testable?

## Output Format

Provide:
- Overall assessment (approve/request changes)
- Specific issues found (with line numbers)
- Suggestions for improvement

Now let's create some sample code to review:

In [ ]:
%%writefile demo_project/src/calculator.py
def calculate(a, b, operation):
    """Perform calculation on two numbers."""
    if operation == 'add':
        return a + b
    elif operation == 'subtract':
        return a - b
    elif operation == 'multiply':
        return a * b
    elif operation == 'divide':
        return a / b  # Potential division by zero!
    else:
        raise ValueError(f"Unknown operation: {operation}")

Stage the file and generate the prompt:

In [ ]:
# Initialize git repo (if not already done)
!cd demo_project && git init 2>/dev/null || true
!cd demo_project && git config user.email "demo@example.com" 2>/dev/null || true
!cd demo_project && git config user.name "Demo User" 2>/dev/null || true

# Stage the file
!cd demo_project && git add src/calculator.py

# Generate the prompt using gtext
!cd demo_project && gtext cast prompts/code-review.md.gtext

print("\n✅ Prompt generated! Check demo_project/prompts/code-review.md")

Let's view the generated prompt:

In [ ]:
with open('demo_project/prompts/code-review.md', 'r') as f:
    prompt = f.read()
    print(prompt)

## Example 2: Composable Prompt Components

Real-world prompts often share common sections. Let's create reusable components.

In [ ]:
%%writefile demo_project/context/project-info.md
## Project Context

**Project**: Demo Calculator
**Language**: Python 3.9+
**Style Guide**: PEP 8
**Testing**: pytest

In [ ]:
%%writefile demo_project/context/recent-commits.md.gtext
## Recent Commits

```include
cli: git log --oneline -5
```

In [ ]:
%%writefile demo_project/prompts/enhanced-review.md.gtext
# Enhanced Code Review

```include
static: context/project-info.md
```

```include
:expand:static: context/recent-commits.md.gtext
```

## Changes Under Review

```include
cli: git diff --cached
```

## Review Task

Please review these changes with the project context in mind.

Note the `:expand:` modifier - it tells gtext to recursively process the included file, expanding the `git log` command.

In [ ]:
# Generate enhanced prompt
!cd demo_project && gtext cast prompts/enhanced-review.md.gtext

# View result
with open('demo_project/prompts/enhanced-review.md', 'r') as f:
    print(f.read())

## Example 3: RAG Pipeline with Code Context

Let's build a more sophisticated RAG prompt that includes:
- Multiple source files
- Test files
- Documentation
- Recent changes

In [ ]:
# Create more files for context
%%writefile demo_project/src/test_calculator.py
import pytest
from calculator import calculate

def test_add():
    assert calculate(2, 3, 'add') == 5

def test_divide():
    assert calculate(10, 2, 'divide') == 5
    # Missing: test for division by zero!

In [ ]:
%%writefile demo_project/prompts/comprehensive-analysis.md.gtext
# Comprehensive Code Analysis Request

You are an AI assistant helping improve code quality.

## Project Files

### Source Code

```include
glob: src/*.py
```

## Recent Activity

```include
cli: git log --oneline --all -10
```

## Analysis Request

Please analyze the code above and provide:

1. **Test Coverage Gaps**: What's not being tested?
2. **Potential Bugs**: Any edge cases or error conditions?
3. **Refactoring Opportunities**: How could the code be improved?
4. **Documentation Needs**: What should be documented better?

Focus on actionable suggestions with specific examples.

In [ ]:
# Generate comprehensive analysis prompt
!cd demo_project && gtext cast prompts/comprehensive-analysis.md.gtext

# View result
with open('demo_project/prompts/comprehensive-analysis.md', 'r') as f:
    prompt = f.read()
    print(prompt)
    print(f"\n📊 Prompt length: {len(prompt)} characters")

## Example 4: Integration with LLM APIs

Now let's see how to integrate gtext-generated prompts with actual LLM APIs.

**Note**: This example shows the pattern. You'll need your own API keys to run it.

In [ ]:
from gtext.processor import TextProcessor
from pathlib import Path

def generate_prompt(template_path: str, output_path: str = None) -> str:
    """Generate prompt from gtext template.
    
    Args:
        template_path: Path to .gtext template file
        output_path: Optional path to save generated prompt
        
    Returns:
        Generated prompt text
    """
    processor = TextProcessor()
    
    # Process template
    template_file = Path(template_path)
    prompt = processor.process_file(template_file, output_path=output_path)
    
    return prompt

# Example: Generate prompt
prompt = generate_prompt('demo_project/prompts/code-review.md.gtext')
print("✅ Prompt generated programmatically!")
print(f"Length: {len(prompt)} characters")

### Integration Pattern: OpenAI

In [ ]:
# Example integration (requires openai package + API key)
def send_to_openai(prompt_template: str, model: str = "gpt-4"):
    """Send gtext-generated prompt to OpenAI.
    
    Args:
        prompt_template: Path to .gtext template
        model: OpenAI model to use
        
    Returns:
        AI response
    """
    # Generate fresh prompt with latest context
    prompt = generate_prompt(prompt_template)
    
    # Send to OpenAI (pseudo-code)
    # import openai
    # response = openai.ChatCompletion.create(
    #     model=model,
    #     messages=[
    #         {"role": "user", "content": prompt}
    #     ]
    # )
    # return response.choices[0].message.content
    
    return "[AI response would appear here]"

print("Pattern demonstrated - add your API key to run!")

### Integration Pattern: Anthropic Claude

In [ ]:
def send_to_claude(prompt_template: str, model: str = "claude-3-opus-20240229"):
    """Send gtext-generated prompt to Anthropic Claude.
    
    Args:
        prompt_template: Path to .gtext template
        model: Claude model to use
        
    Returns:
        AI response
    """
    prompt = generate_prompt(prompt_template)
    
    # Send to Claude (pseudo-code)
    # import anthropic
    # client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    # message = client.messages.create(
    #     model=model,
    #     max_tokens=4096,
    #     messages=[
    #         {"role": "user", "content": prompt}
    #     ]
    # )
    # return message.content[0].text
    
    return "[Claude response would appear here]"

print("Pattern demonstrated - add your API key to run!")

## Example 5: Advanced RAG Patterns

### Pattern 1: Multi-Stage Prompts

Break complex tasks into stages with separate prompts.

In [ ]:
%%writefile demo_project/prompts/stage1-analyze.md.gtext
# Stage 1: Code Analysis

Analyze the following code and identify:
1. Main functionality
2. Dependencies
3. Potential issues

## Code

```include
glob: src/*.py
```

Provide a structured analysis in JSON format.

In [ ]:
%%writefile demo_project/prompts/stage2-improve.md.gtext
# Stage 2: Improvement Suggestions

Based on this analysis:

```include
static: output/analysis-result.json
```

And the original code:

```include
glob: src/*.py
```

Provide specific code improvements with examples.

### Pattern 2: Versioned Prompt Templates

Track prompt evolution in git alongside code changes.

In [ ]:
# Commit prompt templates
!cd demo_project && git add prompts/ context/
!cd demo_project && git commit -m "Add prompt templates v1.0" 2>/dev/null || echo "Already committed"

print("\n✅ Prompt templates versioned in git!")
print("\nBenefits:")
print("- Track what prompts generated what outputs")
print("- A/B test different prompt versions")
print("- Rollback to previous prompts if needed")
print("- Collaborate on prompt engineering")

### Pattern 3: Dynamic Context Selection

In [ ]:
%%writefile demo_project/prompts/smart-context.md.gtext
# Smart Context Selection

## Recently Modified Files (Last 7 Days)

```include
cli: git diff --name-only HEAD~7..HEAD
```

## Active Branch Context

```include
cli: git log origin/main..HEAD --oneline
```

## Most Changed Files (Hot Spots)

```include
cli: git log --pretty=format: --name-only | sort | uniq -c | sort -rg | head -5
```

Use this context to provide targeted analysis.

## Best Practices for RAG with gtext

### 1. Organize Prompts by Purpose

```
prompts/
├── code-review/
│   ├── basic-review.md.gtext
│   ├── security-review.md.gtext
│   └── performance-review.md.gtext
├── documentation/
│   ├── api-docs.md.gtext
│   └── readme-update.md.gtext
└── refactoring/
    ├── suggest-refactor.md.gtext
    └── extract-function.md.gtext
```

### 2. Create Reusable Context Components

```
context/
├── project-info.md         # Static project metadata
├── recent-commits.md.gtext # Dynamic git history
├── code-standards.md       # Coding guidelines
└── test-strategy.md        # Testing approach
```

### 3. Use `:expand:` for Nested Templates

```markdown
```include
:expand:static: context/recent-commits.md.gtext
```
```

This recursively processes ````include` blocks in the included file.

### 4. Measure and Optimize

- Track prompt token counts
- Monitor LLM costs per prompt
- A/B test different prompt structures
- Version prompts alongside code

### 5. Cache Generated Prompts When Appropriate

```python
import hashlib
from pathlib import Path

def cached_generate(template: str, cache_dir: str = '.prompt_cache'):
    """Generate prompt with caching based on template content."""
    template_content = Path(template).read_text()
    cache_key = hashlib.md5(template_content.encode()).hexdigest()
    cache_file = Path(cache_dir) / f"{cache_key}.md"
    
    if cache_file.exists():
        return cache_file.read_text()
    
    # Generate fresh
    prompt = generate_prompt(template)
    
    # Cache it
    cache_file.parent.mkdir(exist_ok=True)
    cache_file.write_text(prompt)
    
    return prompt
```

## Real-World Use Cases

### Use Case 1: Automated Code Review Bot

```bash
# In CI/CD pipeline
git diff origin/main...HEAD > changes.diff
gtext cast prompts/code-review.md.gtext
curl -X POST https://api.openai.com/v1/chat/completions \
  -d @prompts/code-review.md
```

### Use Case 2: Documentation Generation

```python
# Generate API docs from code
prompt = generate_prompt('prompts/generate-api-docs.md.gtext')
docs = llm.generate(prompt)
Path('docs/api.md').write_text(docs)
```

### Use Case 3: Changelog Generation

```markdown
# prompts/changelog.md.gtext

Generate a changelog for these commits:

```include
cli: git log v1.0.0..HEAD --pretty=format:"%h %s"
```

Group by: feat, fix, docs, refactor, test
```

### Use Case 4: Test Generation

```markdown
# prompts/generate-tests.md.gtext

Generate unit tests for:

```include
static: src/new_feature.py
```

Use pytest style, aim for 100% coverage.
```

## Summary

You've learned how to use gtext for RAG and prompt engineering:

✅ **Create composable prompt templates** that include dynamic content

✅ **Use protocol handlers** (`cli:`, `static:`, `glob:`) to gather context

✅ **Apply `:expand:` modifier** for nested template processing

✅ **Integrate with LLM APIs** (OpenAI, Anthropic, etc.)

✅ **Version prompts in git** for reproducibility

✅ **Build RAG pipelines** with reusable components

## Next Steps

- Explore more protocol handlers in the [documentation](https://gtext.readthedocs.io/)
- Check out [future protocols](https://gtext.readthedocs.io/en/latest/extensions/include.html#future-protocols-planned) (`storage:`, `app:`, `db:`)
- Build your own custom extensions
- Share your prompt templates on GitHub!

## Resources

- 📚 [gtext Documentation](https://gtext.readthedocs.io/)
- 🐙 [GitHub Repository](https://github.com/genropy/gtext)
- 🪶 Like a weaverbird, gtext weaves together your context into perfect prompts!

---

**Happy prompt engineering! 🚀**